# Joell Greenblatt's Magic Formula

Value investing approached based on picking wonderful stocks at bargain price:
 - Wonderful stock - ROIC - return on invested capital - look at the core competnecy and see how well they are doing it (i.e. exclude excess cash and interest-bearing assets to focus only on the business assets)
 - Bargain price - is quantified by Earning Yield -> EBIT to Enterprise Value (P/E ratio but capital structure independent)
   - ```Earnings Yield = EBIT / Enterprise Value```
   - ```ROIC = EBIT / (Net Fixed Assets + Net Working Capital)```
   - ```Magic Formula = Rank[Rank(Earnings Yield) + Rank(ROIC)]```
 - Exclude finance and insurance companies
 - Accumulate 2-3 positions per month over a 12-month period, then rebalance each year

In [1]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd

tickers = ["AXP","AAPL","BA","CAT","CVX","CSCO","DIS","DOW", "XOM",
           "HD","IBM","INTC","JNJ","KO","MCD","MMM","MRK","MSFT",
           "NKE","PFE","PG","TRV","UTX","UNH","VZ","V","WMT","WBA"]

#list of tickers whose financial data needs to be extracted
financial_dir = {}


In [2]:
def scrape(tickers):
    financial_dir = {}

    for ticker in tickers:
        try:
        #getting balance sheet data from yahoo finance for the given ticker
            temp_dir = {}
            url = 'https://in.finance.yahoo.com/quote/'+ticker+'/balance-sheet?p='+ticker
            page = requests.get(url)
            page_content = page.content
            soup = BeautifulSoup(page_content,'html.parser')
            tabl = soup.find_all("div", {"class" : "M(0) Mb(10px) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
            for t in tabl:
                rows = t.find_all("div", {"class" : "rw-expnded"})
                for row in rows:
                    temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]

            #getting income statement data from yahoo finance for the given ticker
            url = 'https://in.finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
            page = requests.get(url)
            page_content = page.content
            soup = BeautifulSoup(page_content,'html.parser')
            tabl = soup.find_all("div", {"class" : "M(0) Mb(10px) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
            for t in tabl:
                rows = t.find_all("div", {"class" : "rw-expnded"})
                for row in rows:
                    temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]

            #getting cashflow statement data from yahoo finance for the given ticker
            url = 'https://in.finance.yahoo.com/quote/'+ticker+'/cash-flow?p='+ticker
            page = requests.get(url)
            page_content = page.content
            soup = BeautifulSoup(page_content,'html.parser')
            tabl = soup.find_all("div", {"class" : "M(0) Mb(10px) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
            for t in tabl:
                rows = t.find_all("div", {"class" : "rw-expnded"})
                for row in rows:
                    temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]

            #getting key statistics data from yahoo finance for the given ticker
            url = 'https://in.finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
            page = requests.get(url)
            page_content = page.content
            soup = BeautifulSoup(page_content,'html.parser')
            tabl = soup.findAll("table", {"class": "W(100%) Bdcl(c) Mt(10px) "})
            for t in tabl:
                rows = t.find_all("tr")
                for row in rows:
                    if len(row.get_text(separator='|').split("|")[0:2])>0:
                        temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[-1]    

            #combining all extracted information with the corresponding ticker
            financial_dir[ticker] = temp_dir
        except:
            print("Problem scraping data for ",ticker)

    return financial_dir

In [4]:
if not os.path.exists('data/fin_info.csv'):
    financial_dir = scrape(tickers)
    #storing information in pandas dataframe
    combined_financials = pd.DataFrame(financial_dir)
    combined_financials.dropna(how='all',axis=1,inplace=True) #dropping columns with all NaN values
    tickers = combined_financials.columns #updating the tickers list based on only those tickers whose values were successfully extracted
    for ticker in tickers:
        combined_financials = combined_financials[~combined_financials[ticker].str.contains("[a-z]").fillna(False)]
    combined_financials.to_csv('data/fin_info.csv')
else:
    combined_financials = pd.read_csv('data/fin_info.csv')

In [7]:
# creating dataframe with relevant financial information for each stock using fundamental data
stats = ["EBITDA",
         "Depreciation & amortization",
         "Market cap (intra-day)",
         "Net Income available to common shareholders",
         "Net cash provided by operating activites",
         "Capital expenditure",
         "Total current assets",
         "Total current liabilities",
         "Net property, plant and equipment",
         "Total stockholder equity",
         "Long-term debt",
         "Forward annual dividend yield"] # change as required

indx = ["EBITDA","D&A","MarketCap","NetIncome","CashFlowOps","Capex","CurrAsset",
        "CurrLiab","PPE","BookValue","TotDebt","DivYield"]
all_stats = {}
for ticker in tickers:
    try:
        temp = combined_financials[ticker]
        ticker_stats = []
        for stat in stats:
            ticker_stats.append(temp.loc[stat])
        all_stats['{}'.format(ticker)] = ticker_stats
    except Exception as e:
        print("can't read data for ", ticker, e)


can't read data for  AXP 'EBITDA'
can't read data for  AAPL 'EBITDA'
can't read data for  BA 'EBITDA'
can't read data for  CAT 'EBITDA'
can't read data for  CVX 'EBITDA'
can't read data for  CSCO 'EBITDA'
can't read data for  DIS 'EBITDA'
can't read data for  DOW 'DOW'
can't read data for  XOM 'EBITDA'
can't read data for  HD 'EBITDA'
can't read data for  IBM 'EBITDA'
can't read data for  INTC 'EBITDA'
can't read data for  JNJ 'EBITDA'
can't read data for  KO 'EBITDA'
can't read data for  MCD 'EBITDA'
can't read data for  MMM 'EBITDA'
can't read data for  MRK 'EBITDA'
can't read data for  MSFT 'EBITDA'
can't read data for  NKE 'EBITDA'
can't read data for  PFE 'EBITDA'
can't read data for  PG 'EBITDA'
can't read data for  TRV 'TRV'
can't read data for  UTX 'EBITDA'
can't read data for  UNH 'EBITDA'
can't read data for  VZ 'EBITDA'
can't read data for  V 'V'
can't read data for  WMT 'EBITDA'
can't read data for  WBA 'WBA'


In [ ]:

# cleansing of fundamental data imported in dataframe
all_stats_df = pd.DataFrame(all_stats,index=indx)
all_stats_df[tickers] = all_stats_df[tickers].replace({',': ''}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'M': 'E+03'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'B': 'E+06'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'T': 'E+09'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'%': 'E-02'}, regex=True)
for ticker in all_stats_df.columns:
    all_stats_df[ticker] = pd.to_numeric(all_stats_df[ticker].values,errors='coerce')
all_stats_df.dropna(axis=1,inplace=True)
tickers = all_stats_df.columns


In [ ]:

# calculating relevant financial metrics for each stock
transpose_df = all_stats_df.transpose()
final_stats_df = pd.DataFrame()
final_stats_df["EBIT"] = transpose_df["EBITDA"] - transpose_df["D&A"]
final_stats_df["TEV"] =  transpose_df["MarketCap"].fillna(0) \
                         +transpose_df["TotDebt"].fillna(0) \
                         -(transpose_df["CurrAsset"].fillna(0)-transpose_df["CurrLiab"].fillna(0))
final_stats_df["EarningYield"] =  final_stats_df["EBIT"]/final_stats_df["TEV"]
final_stats_df["FCFYield"] = (transpose_df["CashFlowOps"]-transpose_df["Capex"])/transpose_df["MarketCap"]
final_stats_df["ROC"]  = (transpose_df["EBITDA"] - transpose_df["D&A"])/(transpose_df["PPE"]+transpose_df["CurrAsset"]-transpose_df["CurrLiab"])
final_stats_df["BookToMkt"] = transpose_df["BookValue"]/transpose_df["MarketCap"]
final_stats_df["DivYield"] = transpose_df["DivYield"]


In [ ]:
################################Output Dataframes##############################

# finding value stocks based on Magic Formula
final_stats_val_df = final_stats_df.loc[tickers,:]
final_stats_val_df["CombRank"] = final_stats_val_df["EarningYield"].rank(ascending=False,na_option='bottom')+final_stats_val_df["ROC"].rank(ascending=False,na_option='bottom')
final_stats_val_df["MagicFormulaRank"] = final_stats_val_df["CombRank"].rank(method='first')
value_stocks = final_stats_val_df.sort_values("MagicFormulaRank").iloc[:,[2,4,8]]
print("------------------------------------------------")
print("Value stocks based on Greenblatt's Magic Formula")
print(value_stocks)


# finding highest dividend yield stocks
high_dividend_stocks = final_stats_df.sort_values("DivYield",ascending=False).iloc[:,6]
print("------------------------------------------------")
print("Highest dividend paying stocks")
print(high_dividend_stocks)


# # Magic Formula & Dividend yield combined
final_stats_df["CombRank"] = final_stats_df["EarningYield"].rank(ascending=False,method='first') \
                              +final_stats_df["ROC"].rank(ascending=False,method='first')  \
                              +final_stats_df["DivYield"].rank(ascending=False,method='first')
final_stats_df["CombinedRank"] = final_stats_df["CombRank"].rank(method='first')
value_high_div_stocks = final_stats_df.sort_values("CombinedRank").iloc[:,[2,4,6,8]]
print("------------------------------------------------")
print("Magic Formula and Dividend Yield combined")
print(value_high_div_stocks)
